In [1]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch
set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

n_subs = np.max(params[:,0])

In [3]:
train_sess = {'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'manual','scaler_load':False,'feat_type':'feat','epochs':10}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 2

for sub in range(2,n_subs+1):
    sess.sub = sub
    
    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    if np.sum(train_ind):
        trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)
    
        # Train NNs
        mlp = MLP()
        mlp_beta = MLPbeta()
        cnn = CNN()

        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        test_loss = tf.keras.metrics.Mean(name='test_loss')
        test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

        models = ['mlp','mlpbeta','cnn']

        for model in models:
            print('Training ' + model)
            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()
                test_loss.reset_states()
                test_accuracy.reset_states()

                # Train MLP
                if model == 'mlp':
                    for x, y in trainmlp:
                        train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validmlp:
                        test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
                # Train MLP Beta
                elif model == 'mlpbeta':
                    for x, y in trainmlp:
                        train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validmlp:
                        test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
                # Train CNN
                elif model == 'cnn':
                    for x, y in traincnn:
                        train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
                    for x_test, y_test in validcnn:
                        test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                        f'Test Loss: {test_loss.result():.2f}, '
                        f'Test Accuracy: {test_accuracy.result() * 100:.2f}'
                    )
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        w_cnn, c_cnn,_, _, _ = train_lda(cnn_enc(x_train_cnn).numpy(),y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        with open('real_noise/all_real_noise.p', 'rb') as f:
            real_noise_temp, _ = pickle.load(f)

        test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

        if np.sum(test_ind):
            x_test_cnn, x_test_mlp, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLP ---- '
                f'Clean Accuracy: {clean_acc * 100:.2f}, '
                f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
                f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
                f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLPB ---- '
                f'Clean Accuracy: {clean_acc * 100:.2f}, '
                f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
                f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
                f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'CNN ---- '
                f'Clean Accuracy: {clean_acc * 100:.2f}, '
                f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
                f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
                f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
            )
        else:
            print('no testing data')
    else:
        print('no training data')


Loading training data: traindata_manual/AB2_traindata_2.p
Training mlp
Epoch 1, Loss: 1.936887502670288, Accuracy: 19.05158805847168, Test Loss: 1.9074116945266724, Test Accuracy: 22.22222328186035
Epoch 10, Loss: 1.7667014598846436, Accuracy: 38.519840240478516, Test Loss: 1.7552905082702637, Test Accuracy: 39.412696838378906
Training mlpbeta
Epoch 1, Loss: 1.8376412391662598, Accuracy: 31.13492202758789, Test Loss: 1.7512205839157104, Test Accuracy: 41.396827697753906
Epoch 10, Loss: 1.5570755004882812, Accuracy: 60.83729934692383, Test Loss: 1.556839108467102, Test Accuracy: 60.17460250854492
Training cnn
Epoch 1, Loss: 1.8392361402511597, Accuracy: 30.916667938232422, Test Loss: 1.742846965789795, Test Accuracy: 41.25396728515625
Epoch 10, Loss: 1.421716332435608, Accuracy: 74.68650817871094, Test Loss: 1.492529034614563, Test Accuracy: 66.9047622680664
Loading training data: traindata_manual/AB2_traindata_4.p
MLP ---- Clean Accuracy: 51.77, Noisy Accuracy: 48.69, LDA Clean Accurac